## Численная оптимизация в логистических задачах: ДЗ

Совместим сразу два ДЗ в одно. Пусть есть классическая траспортная задача, которую надо решить двумя способами.

## 0. Генерация задания

In [295]:
import numpy as np

Зададим случайное число складов и магазинов:

In [296]:
N, M = np.random.randint(10, 100, 2)
N, M

(44, 70)

Заполним случайно массивы с количеством товара на складах и запросами магазинов:

**Кстати!** Вы же помните, что суммарное число запасов должно совпадать с суммарными запросами?

In [297]:
storages = np.zeros(N)
demands = np.zeros(M)

In [298]:
import math 

SUM = 100

def get_random_array(size: int, arr_sum: int):
    rand_n = [ np.random.random_sample() for _ in range(size) ]
    result = [ math.floor(i * arr_sum / sum(rand_n)) for i in rand_n ] 
    for _ in range(arr_sum - sum(result)): 
        result[np.random.randint(0, size-1)] += 1
    return np.array(result)

storages = get_random_array(N, SUM)
demands = get_random_array(M, SUM)

assert len(storages) == N
assert len(demands) == M
assert sum(demands) == sum(storages)

In [299]:
storages

array([4, 5, 1, 3, 1, 1, 4, 3, 0, 2, 3, 4, 3, 0, 1, 3, 1, 1, 2, 1, 1, 3,
       1, 1, 0, 1, 3, 5, 5, 3, 4, 3, 0, 3, 2, 0, 5, 1, 2, 4, 3, 2, 4, 1])

In [300]:
demands

array([4, 3, 0, 2, 2, 1, 3, 1, 2, 2, 3, 1, 2, 2, 1, 2, 1, 2, 0, 2, 2, 0,
       1, 3, 1, 2, 2, 2, 1, 2, 2, 1, 0, 1, 1, 0, 0, 0, 1, 0, 2, 0, 4, 3,
       0, 2, 3, 2, 1, 2, 2, 1, 1, 1, 0, 1, 1, 1, 0, 2, 1, 2, 1, 3, 1, 0,
       0, 2, 3, 0])

Сгенерируем случайно матрицу стоимости перевозок:

In [301]:
costs = np.random.randint(1, 20, size=(N, M))


In [302]:
costs

array([[17, 15, 11, ..., 12, 13,  4],
       [ 7, 17, 19, ...,  8,  4,  4],
       [ 4, 14,  4, ..., 17,  1, 13],
       ...,
       [ 2,  8,  8, ..., 14, 14, 11],
       [16,  1, 17, ...,  8,  6,  4],
       [16,  2, 16, ..., 12,  6, 12]])

Фух. Вроде все, можно приступать!

## 1. Постановка ЛП

Напишем честную постановку задачи Линейного программирования (ЛП) и попробуем решить задачу.

In [303]:
from pulp import *
from time import time

In [304]:
def make_problem(storages, demands, costs):
    prob = LpProblem("prob", LpMinimize)

    # выбор пути
    routes = LpVariable.dicts(
        name = "Routes",
        indices = ((i, j) for i in range(len(storages)) for j in range(len(demands))),
        cat = LpBinary
    )

    # целевая функция
    prob += lpSum(costs[(i, j)] * routes[(i,j)] for i in range(len(storages)) for j in range(len(demands)))

    # ограничение на спрос
    for j in range(len(demands)):
        prob += lpSum(routes[(i, j)] for i in range(len(storages))) >= demands[j]

    # ограничение на наличие
    for i in range(len(storages)):
        prob += lpSum(routes[(i, j)] for j in range(len(demands))) <= storages[i] 
    
    return prob

In [305]:
problem = make_problem(storages, demands, costs)

st_time = time()
problem.solve()
print(f"Решение заняло {time() - st_time} сек")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/diemvs/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/n1/x_mxj5qx7mb8bzj93wb8vzl00000gn/T/445c44292477419284cfaad2daf122f4-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/n1/x_mxj5qx7mb8bzj93wb8vzl00000gn/T/445c44292477419284cfaad2daf122f4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 119 COLUMNS
At line 15520 RHS
At line 15635 BOUNDS
At line 18716 ENDATA
Problem MODEL has 114 rows, 3080 columns and 6160 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 163 - 0.00 seconds
Cgl0002I 350 variables fixed
Cgl0004I processed model has 94 rows, 2145 columns (2145 integer (2145 of which binary)) and 4290 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I So

In [306]:
LpStatus[problem.status]

'Optimal'

Посмотрим на значение целевой функции:

In [307]:
problem.objective

17*Routes_(0,_0) + 15*Routes_(0,_1) + 18*Routes_(0,_10) + 4*Routes_(0,_11) + 1*Routes_(0,_12) + 4*Routes_(0,_13) + 6*Routes_(0,_14) + 16*Routes_(0,_15) + 2*Routes_(0,_16) + 7*Routes_(0,_17) + 11*Routes_(0,_18) + 10*Routes_(0,_19) + 11*Routes_(0,_2) + 14*Routes_(0,_20) + 4*Routes_(0,_21) + 1*Routes_(0,_22) + 9*Routes_(0,_23) + 4*Routes_(0,_24) + 7*Routes_(0,_25) + 12*Routes_(0,_26) + 17*Routes_(0,_27) + 4*Routes_(0,_28) + 6*Routes_(0,_29) + 1*Routes_(0,_3) + 16*Routes_(0,_30) + 19*Routes_(0,_31) + 1*Routes_(0,_32) + 12*Routes_(0,_33) + 8*Routes_(0,_34) + 17*Routes_(0,_35) + 5*Routes_(0,_36) + 13*Routes_(0,_37) + 6*Routes_(0,_38) + 6*Routes_(0,_39) + 17*Routes_(0,_4) + 16*Routes_(0,_40) + 8*Routes_(0,_41) + 3*Routes_(0,_42) + 2*Routes_(0,_43) + 11*Routes_(0,_44) + 10*Routes_(0,_45) + 14*Routes_(0,_46) + 19*Routes_(0,_47) + 12*Routes_(0,_48) + 9*Routes_(0,_49) + 16*Routes_(0,_5) + 8*Routes_(0,_50) + 17*Routes_(0,_51) + 2*Routes_(0,_52) + 4*Routes_(0,_53) + 17*Routes_(0,_54) + 9*Routes_(0,

## 2. Метод потенциалов

А теперь давайте для тех же данных реализуем метод потенциалов: будет ли он работать лучше / быстрее?

**Кстати**: для построения начальной конфигурации используйте метод северо-западного угла.

In [308]:
# метод северно-западного угла
def get_init_configuration(d, s):
    N, M = len(s), len(d)
    result = np.zeros((N, M))

    i, j = 0, 0
    
    while i < N and j < M:
        current = min(s[i], d[j])
        result[i, j] = current
        s[i] -= current
        d[j] -= current
        
        if s[i] == 0:
            i += 1
        if d[j] == 0:
            j += 1
            
    return result

storages_test = [10, 20, 30]
demands_test = [15, 20, 25]

expected = np.array([   
    [10,  0,  0],
    [ 5, 15,  0],
    [ 0,  5, 25],
])

assert (get_init_configuration(demands_test, storages_test) == expected).all()

In [309]:
def find_potentials(x, costs):
    N, M = x.shape
    u = np.full(N, np.nan)
    v = np.full(M, np.nan)
    
    # Устанавливаем u[0] = 0 для начала расчета
    u[0] = 0
    

    while np.any(np.isnan(u)) or np.any(np.isnan(v)):
        for i in range(N):
            for j in range(M):
                if x[i, j] > 0:  
                    if np.isnan(u[i]) and not np.isnan(v[j]):
                        u[i] = costs[i, j] - v[j]
                    elif not np.isnan(u[i]) and np.isnan(v[j]):
                        v[j] = costs[i, j] - u[i]
    
    return u, v

In [310]:
def check_optimality(basic_plan, u, v, costs):
    """ Проверка и оптимизация решения """
    N, M = basic_plan.shape
    delta = np.zeros((N, M))  
    
    for i in range(N):
        for j in range(M):
            if basic_plan[i, j] == 0:  
                delta[i, j] = costs[i, j] - (u[i] + v[j])
    

    if (delta >= 0).all():
        return basic_plan, True
    
    for _ in range(10):  
        min_delta_index = np.unravel_index(np.argmin(delta), delta.shape)
        i, j = min_delta_index
        if delta[i, j] >= 0:
            break
    
    return basic_plan, False

**Кстати**: для решения системы уравнений можно использовать фукнкцию np.linalg.solve(A, b), где A - квадратная матрица коэффициентов при переменных в уравнениях, а b - столбец правых частей уравнений. 

In [311]:
def solve_potentials(storages, demands, costs):
    basic_plan = get_init_configuration(d=demands.copy(), s=storages.copy())
    
    optimal = False

    u, v = find_potentials(costs, basic_plan)
        
    while optimal == False:

        delta, optimal = check_optimality(basic_plan, u, v, costs)
        if np.all(delta < 0):  
            u, v = find_potentials(costs, basic_plan) 

    return basic_plan


In [312]:
st_time = time()

res = solve_potentials(storages, demands, costs)

print(f"Решение заняло {time() - st_time} сек")

Решение заняло 0.005017757415771484 сек


Какие выводы можно сделать?

// your ideas here